## View the connectivity matrices in R to check for Liz to move on to more particles, longer releases

In [3]:
Packages <- c("dplyr","lattice", "ggplot2", "dplyr","ncdf4", "tidyr", "stringr", "tidyverse", "raster", "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')

source("~/scripts/conleyte.R")
source("~/scripts/conlabor.R")
labor <- conlabor()
leyte <- conleyte()


In [4]:
#conn_mat <- raster("~/oceanography/ROMS/data/Camotes_Sea_Connectivity_Matrices_08_day_PLD.nc", values=T, band=1)
conn_mat_all <- brick("~/oceanography/ROMS/data/Camotes_Sea_Connectivity_Matrices_08_day_PLD.nc", values=T) 

    

In [5]:
sources <- c("palanas", "wangag", "magbangon", "cabatoan", "caridad_cem", "caridad_prop", "hicgop", "hicgop_south", "sitio_tugas", "el_school", "sitio_lonas", "san_agustin", "psf", "poroc_rose", "pangasugan", "visca", "gabas", "tamakin", "haina", "sitio_baybay", "sandflat1", "sandflat2", "sandflat3", "sandflat4", "sandflat5", "sandflat6", "camotes_islands", "other")
source_ind <- c(seq(1,28, 1))
source_id <- cbind(sources, source_ind)


In [6]:
col <- "date"
col2 <- "index"
season1 <- as.data.frame(seq(as.Date("2010/10/01"), as.Date("2011/05/31"), by="days"))
names(season1) <- col

season2 <- as.data.frame(seq(as.Date("2011/10/01"), as.Date("2012/05/31"), by="days"))
names(season2) <- col

season3 <- as.data.frame(seq(as.Date("2012/10/01"), as.Date("2013/05/31"), by="days"))
names(season3) <- col

season4 <- as.data.frame(seq(as.Date("2013/10/01"), as.Date("2014/04/18"), by="days"))
names(season4) <- col

all_dates <- bind_rows(season1, season2, season3, season4)
time_step <- as.data.frame(seq(1,930,1))
names(time_step)<- col2
all_dates2 <- bind_cols(time_step, all_dates)
dim(all_dates2)


[1] 930   2

In [8]:
head(all_dates2)

index,date
1,2010-10-01
2,2010-10-02
3,2010-10-03
4,2010-10-04
5,2010-10-05
6,2010-10-06


In [22]:
#make each daily connectivity matrix into a df and then pull them all together into a list
conn_mat_list <- vector("list", nrow(all_dates2))

cols <- c("source", "sink", "fraction")

for(i in nrow(all_dates2)){
    
    index_eval <- all_dates2$index[i]
    
    tmp <- as.data.frame(conn_mat_all[[i]], xy=T)   #pull out a data frame for each date
    names(tmp) <- cols
    
    date <- filter(all_dates2, index==index_eval)[2] #grab the date
    date_df <- do.call("rbind", replicate(784, date, simplify = FALSE)) #make a df of the date
    tmp2 <- bind_cols(date_df, tmp)

 
    conn_mat_list[[i]] <- tmp2 
}



In [23]:
head(tmp2)

date,source,sink,fraction
2014-04-18,1,28,0.000000e+00
2014-04-18,2,28,0.000000e+00
2014-04-18,3,28,4.411116e-06
2014-04-18,4,28,2.205558e-06
2014-04-18,5,28,2.205558e-06
2014-04-18,6,28,0.000000e+00


In [146]:
nrow(all_dates)


[1] 930

In [144]:
date

date
2010-10-01
2011-10-01
2012-10-01
2013-10-01


In [99]:
tmp

x,y,fraction,date
1,28,0.000000e+00,16178
2,28,0.000000e+00,16178
3,28,2.205558e-06,16178
4,28,0.000000e+00,16178
5,28,8.822232e-06,16178
6,28,6.616674e-06,16178
7,28,1.764446e-05,16178
8,28,2.205558e-05,16178
9,28,1.102779e-05,16178
10,28,1.102779e-05,16178
